In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Imports

In [2]:
# import necessary libraries
import mesa
import seaborn as sns
import pandas as pd
import shutil

# import cacheable model wrapper
from cacheable_model import CacheableModel

### Output directory 
"output_dir" is the path which the cached data is stored. The directory is included in the .gitignore file. 

In [3]:
directory = "output_dir"

def delete_directory(directory_path):
    shutil.rmtree(directory_path)
    print(f"Directory {directory_path} and its contents have been deleted.")

try:
    delete_directory(directory)
except:
    pass

Directory output_dir and its contents have been deleted.


### Base Model
Define the Boltzmann Wealth Model. I chose to reimplement it here so that I can modify the attributes and experiment with the results more easily. The code is taken from the tutorial.

In [4]:
def compute_gini(model):
    agent_wealths = [agent.wealth for agent in model.schedule.agents]
    x = sorted(agent_wealths)
    N = model.num_agents
    B = sum(xi * (N - i) for i, xi in enumerate(x)) / (N * sum(x))
    return 1 + (1 / N) - 2 * B


class MoneyAgent(mesa.Agent):
    """An agent with fixed initial wealth."""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_money(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        cellmates.pop(
            cellmates.index(self)
        )  # Ensure agent is not giving money to itself
        if len(cellmates) > 1:
            other = self.random.choice(cellmates)
            other.wealth += 1
            self.wealth -= 1
            if other == self:
                print("I JUST GAVE MONEY TO MYSELF HEHEHE!")

    def step(self):
        self.move()
        if self.wealth > 0:
            self.give_money()


class MoneyModel(mesa.Model):
    """A model with some number of agents."""

    def __init__(self, N, width, height):
        super().__init__()
        self.num_agents = N
        self.grid = mesa.space.MultiGrid(width, height, True)
        self.schedule = mesa.time.RandomActivation(self)

        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

        self.datacollector = mesa.DataCollector(
            model_reporters={"Gini": compute_gini}, agent_reporters={"Wealth": "wealth"}
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

### Instantiate the base model and cacheable model

In [5]:
model = MoneyModel(100, 10, 10)

In [6]:
number_of_steps = 1000
def condition_function(model_vars): # condition to cache the results specifically
        return model_vars.get('Gini', 0)[-1] > 0.7
cacheable_model = CacheableModel(model, directory, number_of_steps, 100, condition_function=condition_function)

### Run cacheable model and read cached file

In [7]:
for i in range(number_of_steps):
    cacheable_model.model.step()
    cacheable_model.cache()

In [8]:
model_df, agent_df = cacheable_model.combine_dataframes()
model_df 
agent_df

,Gini
0,0.0000
1,0.3906
2,0.5096
3,0.5974
4,0.5658
...,...
995,0.6628
996,0.6802
997,0.6540
998,0.6692


Wealth
Step AgentID        
0    0             1
     1             1
     2             1
     3             1
     4             1
...              ...
999  95            0
     96            0
     97            3
     98            2
     99            3

[100000 rows x 1 columns]

### Read the critical results dataframe. 
In this simulation, the condition to cache is when gini coefficient > 0.7.

In [17]:
# Replace 'your_file.parquet' with the path to your Parquet file
df = pd.read_parquet('output_dir/special_results.parquet', engine='pyarrow')
df.set_index("Step")


,Gini
Step,
61,0.7014
62,0.7042
64,0.7058
65,0.7110
67,0.7162
...,...
932,0.7054
933,0.7036
934,0.7072
